In [19]:
# !pip install requests beautifulsoup4 pandas

In [20]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [22]:
url = 'https://en.wikipedia.org/wiki/List_of_international_premieral_trips_made_by_Li_Keqiang'

In [23]:
resposne = requests.get(url)
soup = BeautifulSoup(resposne.text, 'html.parser')

In [24]:
tables = soup.find_all('table', {'class': 'wikitable'})

In [25]:
all_data = []

for table in tables:
    headers = [th.get_text(strip=True) for th in table.find_all('th')]
    for row in table.find_all('tr')[1:]:
        cells = row.find_all(['td', 'th'])
        if len(cells) == len(headers):
            data = [cell.get_text(strip=True) for cell in cells]
            record = dict(zip(headers, data))
            all_data.append(record)


In [26]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# url = "https://en.wikipedia.org/wiki/List_of_international_trips_made_by_Xi_Jinping"
# response = requests.get(url)
# soup = BeautifulSoup(response.content, "html.parser")

# all_data = []
# current_year = None


# for tag in soup.find_all(['h2', 'h3', 'table']):
#     if tag.name in ['h2', 'h3']:
#         span = tag.find('span', class_='mw-headline')
#         if span and span.text.strip().startswith('20'):
#             current_year = span.text.strip()
#     elif tag.name == 'table' and 'wikitable' in tag.get('class', []):
#         headers = [th.get_text(strip=True) for th in tag.find_all('th')]
#         for row in tag.find_all('tr')[1:]:
#             cells = row.find_all(['td', 'th'])
#             if len(cells) == len(headers):
#                 values = [cell.get_text(strip=True) for cell in cells]
#                 record = dict(zip(headers, values))
#                 record['Year'] = current_year
#                 all_data.append(record)

# df = pd.DataFrame(all_data)
# df.head()


In [27]:
df = pd.DataFrame(all_data)
df['Name'] = 'Li Keqiang'
df['Level']=  "Former Premier of the State Council of China"

In [28]:
df.head()

,Dates,Country,Locations,Details,Name,Level
0,19–22 May,India,New DelhiMumbai,Official visit.[1]Li held talks with Prime Min...,Li Keqiang,Former Premier of the State Council of China
1,22–23 May,Pakistan,Islamabad,Official visit.[1]Li held talks with President...,Li Keqiang,Former Premier of the State Council of China
2,23–25 May,Switzerland,ZurichBern,Official visit.[1]Li met with the president of...,Li Keqiang,Former Premier of the State Council of China
3,25–27 May,Germany,Berlin,Official visit.[1]Li met with German president...,Li Keqiang,Former Premier of the State Council of China
4,9–11 October,Brunei,Bandar Seri Begawan,Official visit. Li attended the 16th China-ASE...,Li Keqiang,Former Premier of the State Council of China


In [29]:
!pip install country_converter --quiet

In [30]:
import country_converter as coco

In [31]:
df['Continent'] = df['Country'].apply(lambda x: coco.convert(names=x, to='continent'))

In [32]:
developed_countries = {
    "United States", "Canada", "Germany", "France", "United Kingdom",
    "Italy", "Spain", "Japan", "South Korea", "Australia", "New Zealand",
    "Sweden", "Norway", "Finland", "Netherlands", "Denmark", "Switzerland",
    "Austria", "Belgium", "Ireland", "Singapore", "Luxembourg", "Iceland",
    "Israel", "Portugal", "Greece", "Brunei", "Cyprus", "Czech Republic",
    "Estonia", "Hong Kong", "Latvia", "Lithuania", "Malta", "Slovakia",
    "Slovenia", "San Marino", "Taiwan", "United Arab Emirates", "Qatar",
    "Saudi Arabia"
}


communist_countries = {
    "China",
    "Cuba",
    "Vietnam",
    "North Korea",
    "Laos"
}


In [33]:
df['Developed country'] = df['Country'].apply(lambda x:1 if x in developed_countries else 0)
df['Communist country'] = df['Country'].apply(lambda x:1 if x in communist_countries else 0)
df['Developing country'] = df['Country'].apply(lambda x:1 if x not in developed_countries and x not in communist_countries else 0)

In [34]:
df.drop(columns=['Locations'],inplace=True)

df.rename(columns={'Details':'Text'},inplace= True)

In [35]:
df.head()

,Dates,Country,Text,Name,Level,Continent,Developed country,Communist country,Developing country
0,19–22 May,India,Official visit.[1]Li held talks with Prime Min...,Li Keqiang,Former Premier of the State Council of China,Asia,0,0,1
1,22–23 May,Pakistan,Official visit.[1]Li held talks with President...,Li Keqiang,Former Premier of the State Council of China,Asia,0,0,1
2,23–25 May,Switzerland,Official visit.[1]Li met with the president of...,Li Keqiang,Former Premier of the State Council of China,Europe,1,0,0
3,25–27 May,Germany,Official visit.[1]Li met with German president...,Li Keqiang,Former Premier of the State Council of China,Europe,1,0,0
4,9–11 October,Brunei,Official visit. Li attended the 16th China-ASE...,Li Keqiang,Former Premier of the State Council of China,Asia,1,0,0


In [36]:
df.to_excel("/content/drive/MyDrive/GlobalChinaMapping/WikipediaVisits_LiK.xlsx", index=False)